# CBF Tutorial - Part 3: Higher-Order Barrier Functions

Authors: Bardh, Tom and Shakiba

## Message

In this notebook, we consider a nonlinear control affine system. Due to the choice of barrier function and the nonlinearity of the system, we utlize higher-order barrier functions.

The notebook contains,

1. An implementation of a higher order CBFs (HOCBF) for start-to-goal control on a nonlinear system.
2. The formulation of the HOCBF as a quadratic program.
3. A representative example using a nonlinear unicycle model which avoids static obstacles.

## Tutorial

The dynamics of a **[unicycle](http://faculty.salina.k-state.edu/tim/robotics_sg/Control/kinematics/unicycle.html)** can be defined as a time invariant control affine system of the form:

$$
\dot{x} = f(x) + g(x) \cdot u \\
$$

Specifically:

$$
\begin{bmatrix}
    \dot{xr_0}\\
    \dot{xr_1}\\
    \dot{xr_2}
\end{bmatrix} =
\begin{bmatrix}
    cos(xr_2)\\
    sin(xr_2)\\
    0
\end{bmatrix} +
\begin{bmatrix}
    0\\
    0\\
    1
\end{bmatrix} u \tag{1}
$$



where the input to the system $u= \dot{xr_2}$ is the angular velocity.
The system state is the 2D positon and orientation, $\begin{bmatrix} xr_0 & xr_1 & xr_2 \end{bmatrix}^T = \begin{bmatrix} x & y & \theta \end{bmatrix}^T$.
Given an initial position $x^{init}$, our goal is to control this system to a goal location $x^{goal}$.
$$
x^{init} =
    \begin{bmatrix}
        xr^{init}_0\\
        xr^{init}_1
    \end{bmatrix}
\qquad
x^{goal} =
    \begin{bmatrix}
        x^{goal}_0\\
        x^{goal}_1
    \end{bmatrix}
$$
However, the system should avoid certain areas in the workspace (more on this later).

One way to solve this problem is to pose it as an optimization problem where we apply a safety constraint to an existing reference controller which prevents collision to obstacles. Specifically, we would like to solve the following problem:
$$
\min_{u} \quad ||u_{ref} - u|| \\
\textrm{s.t.} \quad \text{CBF constraints to be added later}
$$
So, we will try to minimize the distance between the decission variable $u$ and reference controller $u_{ref}$, while satisfying some constraints. For now, let us disregard the constraint and focus on designing a reference controller.

## Reference controller

We will define a very simple reference controller as follows
$$
u_{ref} = k \cdot (\arctan{(\frac{x^{goal}_0-xr_0}{x^{goal}_1-xr_1})} - xr_2)
$$
This is a two dimensional vector that takes the difference between the current system state (or location) and the goal location and multiply it with a gain factor $k$. This **simple** reference controller will take us from an initial location to the goal location. We can say it is a typical [proportional controller](https://en.wikipedia.org/wiki/Proportional_control). Next, we need to make sure that the system does not enter undesired states.

## Control Barrier Function (CBF) as Constraints

We assume that the undesired regions in the workspace can be defined with one or more [ellipses](https://en.wikipedia.org/wiki/Ellipse). Note that we could also use [hyperellipses](https://mathworld.wolfram.com/Hyperellipse.html) which may be suitable for closely approximating rectangles while maintaining continuity properties. 
In two dimensions, the region inside an ellypsis can be defined with the following inequality
$$
\frac{z^2_0}{a^2} + \frac{z^2_1}{b^2}\leq 1 \tag{2}
$$
where $z_0,z_1$ define the origin and $a,b$ define the major, minor axes of the ellipse, respectively.

In our case, we would like to check whether our system is in the undesired region and therefore the following would enable us to do so:
$$
B(x) = \frac{(x_0-z_0)^2}{a^2} + \frac{(x_1-z_1)^2}{b^2} - 1
$$

Compared to tutorials 1 and 2, we consider a more general definition of CBFs since in this case our system is an affine control system (eq. (1)) whose dynamics are a function of both $f(x)$ and $g(x)$.
(Please see [Wei et al, 2019](https://arxiv.org/pdf/1903.04706.pdf) Definition 4: Control barrier function, Eq. (2). for an overview)
<!--([Ames et al, 2019](https://arxiv.org/pdf/1903.11199.pdf)):-->

Specifcally, if assumptions on the continuity of B(x) are satisfied and there exists a function $\alpha$ such that:

$$
\sup_{u\in U}(L_fB(x) + L_gB(x)u + \alpha(B(x)) \ge 0 \tag{3})
% L_fB(x) + L_gB(x)u + \alpha(B(x)) \ge 0 \tag{7}
$$

is satisfied for all states $x$ in which the system can be in, then $B(x)$ is a valid control barrier function. In other words, if there exists an input which satisfies the constraint in eq. (3) over all possible system states, then $B(x)$ is a valid control barrier function.

A [Lie derivative (see Def 1)](https://arxiv.org/pdf/2105.14311.pdf) is defined as follows: Given a vector field $f$ over $x$, the lie derivative of a polynomial function $B(x)$ along $f$ or order $k\in N$ is

$$
\mathcal{L}^k_fB(x) =
\begin{cases}
    B(x) & k = 0 \\
    \langle \frac{\partial}{\partial x}\mathcal{L}^{k-1}_fB(x),f(x) \rangle & k>0
\end{cases}
$$

Therefore, $L_fB(x) = \langle \frac{\partial}{\partial x} B(x), f(x)\rangle$ and $L_gB(x) = \langle \frac{\partial}{\partial x} B(x), g(x) \rangle$ are the first order [Lie derivatives](https://en.wikipedia.org/wiki/Lie_derivative) of the system. $\langle \cdot,\cdot \rangle$ is a the inner product of vectors. This works well for first order systems, where the control input appears in the first derivative of $B$. However, in our example, this is not the case:

$$
L_gB(x) = 
\begin{bmatrix}
    \frac{2}{a^2}(xr_0 - z_0)\\
    \frac{2}{b^2}(xr_1 - z_1)\\
    0
\end{bmatrix}^T \cdot
\begin{bmatrix}
0 \\
0 \\
1
\end{bmatrix}
= 0
$$

Since the control input does not appear on $L_gB(x)$, we cannot find an input $u$ that satisfies the condition in eq. (3). In fact, as we will see, $B(x)$ has a relative degree of 2 with respect to the system under consideration, since the first time that the control $u$ appears in the derivatives of B along the system dynamics is in its $2^{nd}$ derivative.

For higher order control barrier functions (Please see [Wei et al, 2019](https://arxiv.org/pdf/1903.04706.pdf), Definition 8 Eq. (14)), we rewrite eq. (3) to:

$$
\sup_{u\in U}(L^2_fB(x) + L_gL_fB(x)u + O(\cdot) + \alpha_2( \dot{B}(x) + \alpha_1 B(x) ) \ge 0)
$$

where $O(\cdot)$ denotes the remaining Lie derivatives along $f$ with degree less than or equal to 1. In this case

$$
O(\cdot) = L_fB(x) = \frac{\partial{B}}{\partial{x}}^T \cdot f(x) =
\begin{bmatrix}
    \frac{2}{a^2}(xr_0 - z_0)\\
    \frac{2}{b^2}(xr_1 - z_1)\\
    0
\end{bmatrix}^T
\cdot
\begin{bmatrix}
    cos(xr_2)\\
    sin(xr_2)\\
    0
\end{bmatrix}
$$

In order to evaluate this condition, we have to compute $L^2_fB(x)$ and $L_gL_fB(x)$.

According to the definiton of Lie derivatives:

$$
L^2_fB(x) = \frac{\partial}{\partial x} (L_fB(x)) ^T \cdot f(x)
$$
$$
L_gL_fB(x) = \frac{\partial}{\partial x} (L_fB(x)) ^T \cdot g(x)
$$

<!-- First we compute:
$$
\dot{B}(x) = \frac{\partial{B}}{\partial{x}} \cdot \frac{\partial{x}}{\partial{t}} =
\begin{bmatrix}
    \frac{2}{a^2}(xr_0 - z_0)\\
    \frac{2}{b^2}(xr_1 - z_1)\\
    0
\end{bmatrix}^T
\cdot \dot{x}
$$

(The expansion is similar to [01-DerivativeB.md](./01-DerivativeB.md) in tutorial 1.)
 -->

Then, we calculate
$$
L^2_fB(x) = \frac{2}{a^2}cos^2(xr_2) + \frac{2}{b^2}sin^2(xr_2) \tag{4}
$$

We note that this derivation can (and should) be done with a symbolic math engine such as sympy.

After, we calculate
$$
L_gL_fB(x)u= (-\frac{2}{a^2}(xr_0 - z_0)\cdot sin(xr_2)+\frac{2}{b^2}(xr_1 - z_1)\cdot cos(xr_2))\cdot\dot{xr_2} \tag{5}
$$
(The expansion is shown in [03-LieDerivatives.md](./03-LieDerivatives.md))

Here, we notice that our control input $\dot{xr_2}$ appears in the expression and we can use this control input to satisfy eq. (8). Therefore, we continue in the next step of synthesizing a controller. For more details on higher order barrier functions, please see the paper by [Xiao and Belta](https://arxiv.org/pdf/1903.04706.pdf):

## Posing the control problem as a quadratic program

We can now combine the cost function with the constraints to form an optimization problem.
$$
\begin{align}
\min_{u} \quad & ||u_{ref} - u||\\ \textrm{s.t.} \quad & L_gL_fB(x)u  \ge -L^2_fB(x) - O(\cdot) - \alpha_2( \dot{B}(x) + \alpha_1 B(x) ) \tag{6}
\end{align}
$$

This problem can be posed as a [quadratic problem](https://en.wikipedia.org/wiki/Quadratic_programming) of the form
$$
\begin{align}
\min_{\mathbf{x}} \quad & \frac{1}{2}\mathbf{x}^T P\mathbf{x} + q^T\mathbf{x} \\
\textrm{s.t.} \quad & G\mathbf{x} \le h \tag{7}
\end{align}
$$
In our case, $\mathbf{x} = \dot{x}$ is the decision variable,
Where

$$
P = 1 \tag{8}
$$
$$
q =  u_{ref0} \tag{9}
$$
$$
G = L_gL_fB(x) \tag{10}
$$
$$
h = -L^2_fB(x) - O(\cdot) - \alpha_2( \dot{B}(x) + \alpha_1 B(x) ) \tag{11}
$$

<!-- Here, one of advantage of this methodology is that instance of controller is not needed, as we can see eq. (9) referes only result of controller reference $u_{ref}$. In another word, the CBF can work as an observer in whole controll system. -->

As we can see from eq. (15), this methodology can be utilized with existing controllers. Alternatively, it may be combined with Control Lyapunov Functions as we did in [Yaghoubi et al](https://www.bhoxha.com/papers/LCSS2020.pdf).

## General Remarks

- Selecting appropriate hyperparameters $\alpha_i$ is very important. Bad choices can cause the QP to become infeasible. As they become smaller, the CBF constraint becomes stricter. We can both desing the system with specific hyperparameter values or tune based on outcomes.
- Deadlock may be caused due to symmetry in cases where a robot is going exactly at the center of the obstacle. This can be avoided by adding perturbations to the control input.
- In practical applications, the CBF constraint may be relaxed with a slack variable to avoid issues with feasibility of the problem. In this case, however, you may loose the theoretical guarantess on safety.

## Code implementation

First, we import the necessary packages for our example.

In [1]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
import cvxopt as cvxopt
from matplotlib.patches import Ellipse
from sympy import symbols, Matrix, sin, cos
import matplotlib.animation as animation
from cbflib import cbf, cbf_utils, sys_and_ctrl
from sympy import symbols, Matrix, sin, cos, lambdify, exp, sqrt, log, diff, Mul, srepr, Symbol

In [2]:
# Robot Goal
x_goal = np.array([5, 5])

# Undesired areas in ellipse format (x,y,rad_x,rad_y) - Use example(0) through example(3)
bad_sets = cbf_utils.example(3)

# Parameters for reference controller
ctrl_param = [10]

# Symbols and equations for the CBF
xr0, xr1, xr2, cx, cy, rad_x, rad_y, xr2_dot, u = symbols(
    'xr0 xr1 xr2 cx cy rad_x rad_y xr2_dot u')
symbs = (cx, cy, rad_x, rad_y, xr0, xr1, xr2)

# Barrier function - distance of robot to obstacle
B = ((xr0 - cx)/rad_x)**2 + ((xr1 - cy)/rad_y)**2 - 1

# dx = f(x) + g(x)u
f = Matrix([cos(xr2), sin(xr2), 0])
g = Matrix([0, 0, 1])
states_dot = Matrix([cos(xr2), sin(xr2), xr2_dot])

# Initialize CBF
myCBF = cbf.CBF(B=B, f=f, g=g, symbs = symbs, states=(xr0, xr1, xr2),
                bad_sets=bad_sets, states_dot=states_dot, degree=2,alpha = [10, 10])

#? Simulation settings
T_max = 10
n_samples = 500
T = np.linspace(0, T_max, n_samples)
dt = T[1]-T[0]
params = {'goal_x': x_goal, 'bad_sets': bad_sets,
          'ctrl_param': ctrl_param, 'myCBF': myCBF}

cvxopt.solvers.options['show_progress'] = False

x_0 = np.array([0.5, 0.5, 0])

In [3]:
# Loop through initial conditions
print('\nComputing trajectories for the initial condition:')
print('x_0\t x_1')
print(x_0[0], '\t', x_0[1])

# If initial condition is inside the bad set, error out.
curr_bs = []
for idxj, j in enumerate(bad_sets):
    curr_bs = bad_sets[idxj]
    assert cbf_utils.is_inside_ellipse(
        [x_0[0], x_0[1]], bad_sets[idxj]) == 0, "Initial condition is inside ellipse"


# Compute output on the unicycle system for given initial conditions and timesteps T
x = np.zeros((np.size(x_0), len(T)))
x[:, 0] = x_0
for i in range(len(T)-1):
    x[:, i+1] = x[:, i] + dt * \
        np.array(sys_and_ctrl.unicycle_f(T[i], x[:, i], [], params))


Computing trajectories for the initial condition:
x_0	 x_1
0.5 	 0.5


In [4]:
# Animate
fig, ax = plt.subplots(figsize=(8,8))
ax = cbf_utils.plot_cbf_elements(ax, bad_sets, x_goal)

plt.xlim(-2, 7)
plt.ylim(-2, 7)

line1, = ax.plot([], [], lw=2)
goal_square = plt.Rectangle(
    x_goal-np.array([.5, .5]), .2, .2, color='r', alpha=0.5)

def init():
    line1.set_data([], [])
    return line1


def animate(i):
    line1.set_data((x[0][0:i], x[1][0:i]))
    return line1, goal_square

plt.close()

ani = animation.FuncAnimation(
    fig, animate, interval=60, frames=n_samples, repeat=True, blit=True)

from IPython.display import HTML
from matplotlib import rcParams
rcParams['animation.html'] = 'html5'
HTML(ani.to_html5_video())
ani


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

To run the code directly in python see [03-Tutorial-HOCFB-Static-Obstacles.py](./03-Tutorial-HOCFB-Static-Obstacles.py)